In [9]:
import spotipy
import time
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os

In [10]:
# Cargar variables del archivo .env
load_dotenv()

# Leer las variables
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

In [11]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [12]:
# Ejemplo: buscar un artista
results = sp.search(q='The Weeknd', type='artist')


In [13]:
# Raw data 
results

{'artists': {'href': 'https://api.spotify.com/v1/search?offset=0&limit=10&query=The%20Weeknd&type=artist',
  'limit': 10,
  'next': 'https://api.spotify.com/v1/search?offset=10&limit=10&query=The%20Weeknd&type=artist',
  'offset': 0,
  'previous': None,
  'total': 822,
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
    'followers': {'href': None, 'total': 104797623},
    'genres': [],
    'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
    'id': '1Xyo4u8uXC1ZmMpatF05PJ',
    'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb9e528993a2820267b97f6aae',
      'height': 640,
      'width': 640},
     {'url': 'https://i.scdn.co/image/ab676161000051749e528993a2820267b97f6aae',
      'height': 320,
      'width': 320},
     {'url': 'https://i.scdn.co/image/ab6761610000f1789e528993a2820267b97f6aae',
      'height': 160,
      'width': 160}],
    'name': 'The Weeknd',
    'popularity': 97,
    'type': 'artist',

In [6]:
artist = results['artists']['items'][0]
print(f"Nombre del artista: {artist['name']}")
print(f"Popularidad: {artist['popularity']}")
print(f"Géneros: {artist['genres']}")

Nombre del artista: The Weeknd
Popularidad: 97
Géneros: []


In [7]:
# Ejemplo: obtener las canciones top de un artista
top_tracks = sp.artist_top_tracks(artist['id'])
print("\nCanciones más populares:")
for i, track in enumerate(top_tracks['tracks'][:5]):
    print(f"{i+1}. {track['name']}")


Canciones más populares:
1. Timeless (feat Playboi Carti)
2. One Of The Girls (with JENNIE, Lily Rose Depp)
3. Starboy
4. Blinding Lights
5. Die For You


In [8]:
redirect_uri = "http://127.0.0.1:8888/callback"  # Puede ser cualquier URL, incluso si no existe

# Inicializar Spotify con OAuth
# El scope 'user-read-recently-played' es necesario para acceder al historial
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri=redirect_uri,
    scope="user-read-recently-played",
    cache_path=".spotify_cache"  # Guarda el token para no tener que autenticarte cada vez
))


In [21]:
# Obtener las reproducciones recientes (máximo 50)
recent_tracks = sp.current_user_recently_played(limit=50)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [20]:


# Crear una lista para almacenar la información
tracks_list = []

# Procesar cada track
for item in recent_tracks['items']:
    track = item['track']
    played_at = item['played_at']
    
    # Convertir la fecha de ISO 8601 a formato más legible
    played_at_formatted = time.strftime(
        '%Y-%m-%d %H:%M:%S', 
        time.strptime(played_at, '%Y-%m-%dT%H:%M:%S.%fZ')
    )
    
    # Obtener artistas (puede haber varios)
    artists = ", ".join([artist['name'] for artist in track['artists']])
    
    # Añadir a la lista
    tracks_list.append({
        'Reproducido el': played_at_formatted,
        'Canción': track['name'],
        'Artista(s)': artists,
        'Álbum': track['album']['name'],
        'Duración (ms)': track['duration_ms'],
        'Popularidad': track['popularity']
    })

# Convertir a DataFrame para mejor visualización
df = pd.DataFrame(tracks_list)

# Mostrar las reproducciones recientes
print(f"Tus {len(tracks_list)} reproducciones más recientes:")
print(df[['Reproducido el', 'Canción', 'Artista(s)', 'Álbum']])

# Opcionalmente, guardar en un CSV
df.to_csv('mis_reproducciones_recientes.csv', index=False)
print("\nDatos guardados en 'mis_reproducciones_recientes.csv'")

Tus 50 reproducciones más recientes:
         Reproducido el                          Canción  \
0   2025-05-13 03:46:02                  Solo Me Dejaste   
1   2025-05-13 03:42:55            Pongámonos De Acuerdo   
2   2025-05-13 03:40:13                      LA PATRULLA   
3   2025-05-13 03:38:02                          La Nena   
4   2025-05-13 03:35:06             Si Te Pudiera Mentir   
5   2025-05-13 03:30:50  El Mayor de los Ranas (En Vivo)   
6   2025-05-13 03:27:55               El Amor De Tu Vida   
7   2025-05-13 03:25:11                   Lo Tienes Todo   
8   2025-05-13 03:22:26     El Final De Nuestra Historia   
9   2025-05-13 03:19:11                            Chale   
10  2025-05-13 03:16:19                   Si Fuera Fácil   
11  2025-05-12 17:57:08               Siempre Pendientes   
12  2025-05-11 03:32:02                          Que Sea   
13  2025-05-11 03:29:11                  Solo Me Dejaste   
14  2025-05-11 03:26:04            Pongámonos De Acuerdo   
15 